In [40]:
import folium
from function import folium_functions
import function
import pandas as pd

global_path = '../data/London_in_depth/'
output_map_name = global_path + "london_tubes.html"

# Load data

In [2]:
# Files containing the position of the London underground stations and lines
filename_stations = global_path + 'london_stations.csv'
filename_lines = global_path + 'london_tube_lines.csv'

# Initialize the map

In [3]:
city_of_london_coordinates = [51.51279, -0.09184]
m = folium.Map(location=city_of_london_coordinates, zoom_start=12)
m

# Add the stations

In [4]:
dic_stations = folium_functions.pipeline_stations(filename_stations, max_zone=2, current_map=m)
m

# Add the lines

In [5]:
folium_functions.pipeline_lines(filename_lines, filename_stations, m)
m

# 1) Studies of entry and exit

In [37]:
# Load data
df = pd.read_csv('/home/louis/summer_invitational_data_open_2020/data/London/london_underground_activity.csv')
df = df[df['year'] == 2012]
df = df[['year', 'station', 'area', 'annual_entry_and_exit']]

# Remove City of London
df = df[df['area'] != 'City of London']

# Normalize annual_entry_and_exit
df['annual_entry_and_exit'] = df['annual_entry_and_exit'] / df['annual_entry_and_exit'].sum()

# Get area data
df = df.groupby('area')['annual_entry_and_exit'].mean().to_frame().reset_index()

In [42]:
# Plot the new map
m = function.folium_functions.get_raw_map()
path_london_borough = function.get_path_london_boroughs(global_path)
folium.Choropleth(geo_data=path_london_borough, name='choropleth', data=df, 
                  columns=['area', 'annual_entry_and_exit'],
                  key_on='feature.properties.NAME', fill_color='YlOrRd', fill_opacity=0.7, 
                  legend_name='Percent of the yearly entry and exit (underground)', 
                  line_opacity=0.2).add_to(m)
m